In [1]:
# ''' 
# Goto config ->config.yaml
# Paste below code

# data_validation:
#   root_dir: artifacts/data_validation
#   STATUS_FILE: artifacts/data_validation/status.txt
#   unzip_data_dir: artifacts/data_ingestion/winequality-red.csv

# '''

In [2]:
import os

In [3]:
%pwd

'd:\\Project\\Wine_Quality_MLflow_AWS_CICD_Pipeline_End_to_End_Deployment\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Project\\Wine_Quality_MLflow_AWS_CICD_Pipeline_End_to_End_Deployment'

In [6]:
import pandas as pd
data=pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
data.shape

(1599, 12)

In [10]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

# Entity Folder
1. Goto entity folder  src-> textSummarizer -> entity-> __init__.py 
2. Paste below code in __init__.py constructor file

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir:Path
    all_schema: dict

# Config folder 
1. Copy get_data_validation_config function 
2. Goto config folder src->wine_quality->config->configuration.py paste under ConfigurationManager class
3. Import DataValidationConfig from wine_quality.entity using this line  
from wine_quality.entity import DataValidationConfig

In [12]:
from wine_quality.constants import *
from wine_quality.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])


    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config
    

# Component folder
1. Copy entire DataValiadtion class
2. Goto Component folder [ src-> wine_quality -> components - > data_validation.py ]
2. Paste DataValiadtion class in data_validation.py
3. Import DataValidationConfig from entity using this line 

from wine_quality.entity import DataValidationConfig

In [14]:
import os
from wine_quality import logger

In [15]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

# Pipeline folder

1. Goto Component folder [ src-> wine_quality -> pipeline - > stage_02_data_validation.py ]

2. Paste below code in stage_02_data_validation.py

In [16]:
# ''' 
# from wine_quality.logging import logger
# from wine_quality.config.configuration import ConfigurationManager
# from wine_quality.components.data_validation import DataValidation
# class DataValidationTrainingPipeline:
#     def __init__(self):
#         pass
#     def main(self):
        
# '''

3. Copy try block code 
4. Paste in main function in stage_02_data_validation.py

5. Import dataIngestionconfig from config.configuration using this line  
from wine_quality.config.configuration import ConfigurationManager

6. Import data_validation from  components.data_validation using this line  
from wine_quality.components.data_validation import DataValidation

In [17]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-09-17 19:39:58,960] - wine_quality - INFO - common file - LineNum:27   - yaml file : config\config.yaml loaded successfully
[2023-09-17 19:39:58,967] - wine_quality - INFO - common file - LineNum:27   - yaml file : params.yaml loaded successfully
[2023-09-17 19:39:58,978] - wine_quality - INFO - common file - LineNum:27   - yaml file : schema.yaml loaded successfully
[2023-09-17 19:39:58,984] - wine_quality - INFO - common file - LineNum:43   - Created Directory At: artifacts
[2023-09-17 19:39:58,987] - wine_quality - INFO - common file - LineNum:43   - Created Directory At: artifacts/data_validation


In [18]:
# '''
# Example 
# from wine_quality.components.data_validation import DataValidation
# class DataValidationTrainingPipeline:
#     def __init__(self):
#         pass
#     def main(self):
#         config = ConfigurationManager()
#         data_validation_config = config.get_data_validation_config()
#         data_validation = DataValidation(config=data_validation_config)
#         data_validation.validate_all_columns()

# '''

In [19]:
# '''
# 1. Goto main.py and paste below code

# from wine_quality.pipeline.stage_02_data_validation import DataValidationTrainingPipeline
# STAGE_NAME = "Data Validation stage"
# try:
#    logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<") 
#    data_validation = DataValidationTrainingPipeline()
#    data_validation.main()
#    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
# except Exception as e:
#         logger.exception(e)
#         raise e

# 2. Run main.py 
# '''